In [1]:
%pip install bs4

  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import bs4
import re
with open("Fri_01_Nov_2019_12_00_40_.html", "r", encoding="utf-8") as file:
    html = file.read()


FileNotFoundError: [Errno 2] No such file or directory: 'Fri_01_Nov_2019_12_00_40_.html'

Commande

In [4]:
import bs4
import re
from bs4 import BeautifulSoup

def ExtractionDonnee( Fichier):
    with open(str(Fichier), "r", encoding="utf-8") as file:
        html = file.read()


    soup = BeautifulSoup(html, "html.parser")
    td = soup.find("td", {"valign": "top"})
    ps = td.find_all("p")
    h2 = td.find_all("h2")

    #for elt in h2:
        #print(elt.get_text(strip=True))
    Numero_commande = h2[3].get_text().strip().lower()
    Numero_commande = (int(Numero_commande.replace(' ','').replace('\n','').replace('voicivotrereçucommanden°','').strip()))
    #print(Nom,Numero_commande)

    if len(ps) >= 11:
        NomR = ps[1].get_text(strip=True)
        AdresseR = ps[2].get_text(strip=True)
        VilleR = ps[3].get_text(strip=True)
        Code_postalR = ps[4].get_text(strip=True)
        TelephoneR = ps[5].get_text(strip = True)
        Nom = ps[6].get_text(strip = True)
        AdresseC = ps[7].get_text(strip = True)
        VilleC = ps[8].get_text(strip = True)
        Code_PostalC = ps[9].get_text(strip = True)
        TelephoneC = ps[10].get_text(strip = True)
        #print(len(ps))
        #for elt in ps :
            #print(elt.get_text(strip=True))
    Liste_Commande = []
    Liste_elt = []
    i = 5
    com = False
    fees = False
    total = False
    Frais = ""
    Total = ""
    while i < len (ps):
        if re.search(r'\d+\s*x', ps[i].get_text().strip().lower()):
            com = True
        if com == True :
            Liste_elt.append(ps[i].get_text().strip().lower())
        if re.search(r'\d+\s*€', ps[i].get_text().strip().lower()) and com == True:
            com = False
            Liste_Commande.append(Liste_elt)
            Liste_elt = []
        if fees == True :
            fees = False
            Frais = ps[i].get_text().strip().lower()
        if 'Frais de livraison' in ps[i].get_text().strip() :
            fees = True
        if total == True :
            total = False
            Total = ps[i].get_text().strip().lower()
        if 'Total' in ps[i].get_text().strip() :
            total = True
        i+=1
    #print(Liste_Commande)
    for elt in Liste_Commande :
        elt[-1] = float(elt[-1].replace('\xa0','').replace(',','.').replace('€','').strip())
        elt[0] = float(elt[0].replace('x','').strip())
    Total=Total.replace('\xa0','').replace(',','.').replace('€','').strip()
    Frais=Frais.replace('\xa0','').replace(',','.').replace('€','').strip()

    Data = {"Order": {"order_number": Numero_commande, "delivery_fees" : Frais,"order_total_paid":Total}, "restaurant":{"name":NomR,"address" : AdresseR,"city":VilleR,"postcode":Code_postalR,"phone number" : TelephoneR}, "Customer":{"name" : Nom, "address": AdresseC,"city" : VilleC, "postcode" : Code_PostalC, "phone_number":TelephoneC},"order_item" : []}
    for elt in Liste_Commande :
        Data["order_item"].append({"name":elt[1],"quantity":int(elt[0]),"price":elt[-1]})

    return Data
    #print("AdresseR :", AdresseR)
    #print("VilleR :", VilleR)
    #print("Code postalR :", Code_postalR)
    #print("commande :", Liste_Commande)
    #print("TéléphoneR : ", TelephoneR)
    #print("AdresseC :", AdresseC)
    #print("VilleC :", VilleC)
    #print("Code postal C :", Code_PostalC)
    #print("TéléphoneC : ", TelephoneC)
    #print('Numero_commande', Numero_commande)
    #print('Frais de livraison', Frais)
    #print('Total' , Total)




In [6]:
ExtractionDonnee('Fri_01_Nov_2019_12_00_40_.html')

{'Order': {'order_number': 2662,
  'delivery_fees': '3.19',
  'order_total_paid': '36.39'},
 'restaurant': {'name': 'Le Pain Quotidien',
  'address': '8 Rue de Bretagne',
  'city': 'Paris',
  'postcode': '75003',
  'phone number': '+33144780895'},
 'Customer': {'name': 'Regis amichia',
  'address': '20 Rue Saint-Ambroise',
  'city': 'Paris',
  'postcode': '75011PARIS',
  'phone_number': '+33 6 15 78 51 52'},
 'order_item': [{'name': 'tartine jambon fumé & mozzarella di bufala d.o.p bio',
   'quantity': 1,
   'price': 7.9},
  {'name': 'petit-déjeuner 1 personne', 'quantity': 1, 'price': 10.4},
  {'name': 'petit-déjeuner 1 personne', 'quantity': 1, 'price': 10.4},
  {'name': 'mini salade lentilles au paprika fumé (100% végétarien)',
   'quantity': 1,
   'price': 4.5}]}

In [7]:
import os

dossier = "C:\\Users\\bouil\\Desktop\\HoD 101\\albert-hod-101-2026-11\\exercice1_scraping"
Fichiers = os.listdir(dossier)
ListeDesData = []
for elt in Fichiers :
    chemin_complet = os.path.join(dossier, elt)
    ListeDesData.append(ExtractionDonnee(chemin_complet))
ListeDesData

[{'Order': {'order_number': 2662,
   'delivery_fees': '3.19',
   'order_total_paid': '36.39'},
  'restaurant': {'name': 'Le Pain Quotidien',
   'address': '8 Rue de Bretagne',
   'city': 'Paris',
   'postcode': '75003',
   'phone number': '+33144780895'},
  'Customer': {'name': 'Regis amichia',
   'address': '20 Rue Saint-Ambroise',
   'city': 'Paris',
   'postcode': '75011PARIS',
   'phone_number': '+33 6 15 78 51 52'},
  'order_item': [{'name': 'tartine jambon fumé & mozzarella di bufala d.o.p bio',
    'quantity': 1,
    'price': 7.9},
   {'name': 'petit-déjeuner 1 personne', 'quantity': 1, 'price': 10.4},
   {'name': 'petit-déjeuner 1 personne', 'quantity': 1, 'price': 10.4},
   {'name': 'mini salade lentilles au paprika fumé (100% végétarien)',
    'quantity': 1,
    'price': 4.5}]},
 {'Order': {'order_number': 9098,
   'delivery_fees': '2.69',
   'order_total_paid': '34.59'},
  'restaurant': {'name': "Starvin' Joe",
   'address': '42 Rue de Charonne',
   'city': 'Paris',
   'pos

In [95]:
import csv

# Vos données (remplacez par votre liste de commandes)
ListeDesData = ListeDesData  # Votre liste de commandes ici

# Chemin du fichier CSV de sortie
fichier_csv = "commandes.csv"

# Définir les colonnes du CSV
colonnes = [
    "order_number", "delivery_fees", "order_total_paid",
    "restaurant_name", "restaurant_address", "restaurant_city", "restaurant_postcode", "restaurant_phone",
    "customer_name", "customer_address", "customer_city", "customer_postcode", "customer_phone",
    "item_name", "item_quantity", "item_price"
]

# Écrire dans le fichier CSV
with open(fichier_csv, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=colonnes)
    writer.writeheader()  # Écrire l'en-tête

    for commande in ListeDesData:
        # Extraire les informations de la commande
        order_info = commande["Order"]
        restaurant_info = commande["restaurant"]
        customer_info = commande["Customer"]

        # Parcourir chaque article de la commande
        for item in commande["order_item"]:
            row = {
                "order_number": order_info["order_number"],
                "delivery_fees": order_info["delivery_fees"],
                "order_total_paid": order_info["order_total_paid"],
                "restaurant_name": restaurant_info["name"],
                "restaurant_address": restaurant_info["address"],
                "restaurant_city": restaurant_info["city"],
                "restaurant_postcode": restaurant_info["postcode"],
                "restaurant_phone": restaurant_info["phone number"],
                "customer_name": customer_info["name"],
                "customer_address": customer_info["address"],
                "customer_city": customer_info["city"],
                "customer_postcode": customer_info["postcode"],
                "customer_phone": customer_info["phone_number"],
                "item_name": item["name"],
                "item_quantity": item["quantity"],
                "item_price": item["price"]
            }
            writer.writerow(row)

print(f"Le fichier CSV a été généré : {fichier_csv}")


Le fichier CSV a été généré : commandes.csv
